In [1]:
# Test reconstruction performance

In [2]:
import os
import random
import warnings
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')

from keras.optimizers import Adam
from models import AE
# from models import USAD
from models import USAD_keras
from utils import *

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)
seed_everything(42)

# AutoEncoder(AE)
- LSTM-AE
- GRU-AE
- TCN-AE

In [3]:
# make dataset
df = pd.read_csv('./datasets/JNU_elec.csv').set_index('timestamp')
seq_data = create_seq(df, seq_len=24)

# split(train | val | test)
train = seq_data[:(365 * 24 * 3)]
val = seq_data[(365 * 24 * 3):(365 * 24 * 4)]
test = seq_data[(365 * 24 * 4):]

print(f'Train: {train.shape}')
print(f'Val: {val.shape}')
print(f'Test: {test.shape}')

Train: (26280, 24, 1)
Val: (8760, 24, 1)
Test: (3601, 24, 1)


In [4]:
# Min-Max norm
min_val, max_val = np.min(train), np.max(train)
train_norm = minmax(train, min_val, max_val)
val_norm = minmax(val, min_val, max_val)
test_norm = minmax(test, min_val, max_val)

# make tf dataset
train_dataset = make_dataset(train_norm, train_norm)
val_dataset = make_dataset(val_norm, val_norm)
test_dataset = make_dataset(test_norm, test_norm)

In [5]:
# LSTM-AE
lstmae = AE(mode='LSTM')
# GRU-AE
gruae = AE(mode='GRU')
# TCN-AE
tcnae = AE(z_dim=32, hidden_dim=32, dilations=[1, 2, 4], mode='TCN')

input_shape = (None, train.shape[1], train.shape[2])
lstmae.build(input_shape)
gruae.build(input_shape)
tcnae.build(input_shape)

lstmae.compile(optimizer=Adam(.001), loss='mse', metrics='mae')
gruae.compile(optimizer=Adam(.001), loss='mse', metrics='mae')
tcnae.compile(optimizer=Adam(.001), loss='mse', metrics='mae')

lstm_time = TimeHistory()
gru_time = TimeHistory()
tcn_time = TimeHistory()

In [6]:
print(lstmae.summary(), gruae.summary(), tcnae.summary())

Model: "ae"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder (Encoder)           multiple                  16896     
                                                                 
 decoder (Decoder)           multiple                  115457    
                                                                 
Total params: 132,353
Trainable params: 132,353
Non-trainable params: 0
_________________________________________________________________
Model: "ae_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_1 (Encoder)         multiple                  12864     
                                                                 
 decoder_1 (Decoder)         multiple                  91137     
                                                                 
Total params: 104,001
Trainable params: 104,

In [7]:
epochs = 50
batch_size = 64

lstmae.fit(train_dataset, epochs=epochs, batch_size=batch_size, validation_data=(val_dataset), callbacks=[lstm_time])
print(f'LSTM-AE train time: {sum(lstm_time.times):.4f}')

Epoch 1/50
411/411 [==============================] - 5s 8ms/step - loss: 0.0107 - mae: 0.0704 - val_loss: 0.0053 - val_mae: 0.0491
Epoch 2/50
411/411 [==============================] - 3s 7ms/step - loss: 0.0037 - mae: 0.0418 - val_loss: 0.0045 - val_mae: 0.0441
Epoch 3/50
411/411 [==============================] - 3s 7ms/step - loss: 0.0029 - mae: 0.0364 - val_loss: 0.0039 - val_mae: 0.0407
Epoch 4/50
411/411 [==============================] - 3s 7ms/step - loss: 0.0026 - mae: 0.0341 - val_loss: 0.0035 - val_mae: 0.0385
Epoch 5/50
411/411 [==============================] - 3s 8ms/step - loss: 0.0024 - mae: 0.0323 - val_loss: 0.0032 - val_mae: 0.0364
Epoch 6/50
411/411 [==============================] - 3s 8ms/step - loss: 0.0023 - mae: 0.0317 - val_loss: 0.0031 - val_mae: 0.0363
Epoch 7/50
411/411 [==============================] - 3s 8ms/step - loss: 0.0019 - mae: 0.0291 - val_loss: 0.0028 - val_mae: 0.0329
Epoch 8/50
411/411 [==============================] - 3s 7ms/step - loss: 0.

In [8]:
gruae.fit(train_dataset, epochs=epochs, batch_size=batch_size, validation_data=(val_dataset), callbacks=[gru_time])
print(f'GRU-AE train time: {sum(gru_time.times):.4f}')

Epoch 1/50
411/411 [==============================] - 5s 8ms/step - loss: 0.0225 - mae: 0.1128 - val_loss: 0.0158 - val_mae: 0.0868
Epoch 2/50
411/411 [==============================] - 3s 7ms/step - loss: 0.0059 - mae: 0.0522 - val_loss: 0.0052 - val_mae: 0.0484
Epoch 3/50
411/411 [==============================] - 3s 7ms/step - loss: 0.0040 - mae: 0.0423 - val_loss: 0.0048 - val_mae: 0.0458
Epoch 4/50
411/411 [==============================] - 3s 7ms/step - loss: 0.0033 - mae: 0.0388 - val_loss: 0.0039 - val_mae: 0.0415
Epoch 5/50
411/411 [==============================] - 3s 7ms/step - loss: 0.0027 - mae: 0.0344 - val_loss: 0.0038 - val_mae: 0.0394
Epoch 6/50
411/411 [==============================] - 3s 7ms/step - loss: 0.0024 - mae: 0.0322 - val_loss: 0.0036 - val_mae: 0.0388
Epoch 7/50
411/411 [==============================] - 3s 7ms/step - loss: 0.0023 - mae: 0.0315 - val_loss: 0.0034 - val_mae: 0.0372
Epoch 8/50
411/411 [==============================] - 3s 7ms/step - loss: 0.

In [9]:
tcnae.fit(train_dataset, epochs=epochs, batch_size=batch_size, validation_data=(val_dataset), callbacks=[tcn_time])
print(f'TCN-AE train time: {sum(tcn_time.times):.4f}')

Epoch 1/50


411/411 [==============================] - 4s 8ms/step - loss: 0.0084 - mae: 0.0592 - val_loss: 0.0027 - val_mae: 0.0341
Epoch 2/50
411/411 [==============================] - 3s 7ms/step - loss: 0.0017 - mae: 0.0279 - val_loss: 0.0014 - val_mae: 0.0254
Epoch 3/50
411/411 [==============================] - 3s 7ms/step - loss: 0.0011 - mae: 0.0223 - val_loss: 0.0015 - val_mae: 0.0255
Epoch 4/50
411/411 [==============================] - 3s 7ms/step - loss: 7.3599e-04 - mae: 0.0184 - val_loss: 9.9651e-04 - val_mae: 0.0202
Epoch 5/50
411/411 [==============================] - 3s 7ms/step - loss: 5.7291e-04 - mae: 0.0162 - val_loss: 9.2520e-04 - val_mae: 0.0207
Epoch 6/50
411/411 [==============================] - 3s 7ms/step - loss: 4.5104e-04 - mae: 0.0146 - val_loss: 5.8347e-04 - val_mae: 0.0158
Epoch 7/50
411/411 [==============================] - 3s 7ms/step - loss: 3.1101e-04 - mae: 0.0120 - val_loss: 4.1752e-04 - val_mae: 0.0136
Epoch 8/50
411/411 [==============================] - 3

# UnSupervisedAnomalyDetection(USAD)
- USAD
- TCN-USAD

In [10]:
# USAD
# usad = USAD(hidden_dims=[128, 64], mode='USAD')
usad = USAD_keras(z_dim=32, hidden_dims=[64, 32], mode='USAD')
# TCN-USAD
# tcnusad = USAD(z_dim=32, hidden_dims=32, dilations=[1, 2, 4], mode='TCN')
tcnusad = USAD_keras(z_dim=32, hidden_dims=64, dilations=[1, 2, 4], mode='TCN')

input_shape = (None, train.shape[1], train.shape[2])
usad.build(input_shape)
tcnusad.build(input_shape)

usad.compile(
    g_optimizer=Adam(.001), d_optimizer=Adam(.001)
)
tcnusad.compile(
    g_optimizer=Adam(.001), d_optimizer=Adam(.001)
)

usad_time = TimeHistory()
tcnusad_time = TimeHistory()

In [11]:
print(usad.summary(), tcnusad.summary())

Model: "usad_keras"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_3 (Encoder)         multiple                  3264      
                                                                 
 decoder_3 (Decoder)         multiple                  3233      
                                                                 
 decoder_4 (Decoder)         multiple                  3233      
                                                                 
Total params: 9,738
Trainable params: 9,730
Non-trainable params: 8
_________________________________________________________________
Model: "usad_keras_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_4 (Encoder)         multiple                  90656     
                                                                 
 decoder_5 (Decoder)         mul

In [12]:
# usad_times, _ = usad.fit(train_dataset, val_dataset, epochs)
# print(f'USAD train time: {sum(usad_times):.4f}')
usad.fit(train_dataset, epochs=epochs, batch_size=batch_size, validation_data=(val_dataset), callbacks=[usad_time])
print(f'USAD train time: {sum(usad_time.times):.4f}')

Epoch 1/50
411/411 [==============================] - 7s 15ms/step - g_loss: 0.0230 - d_loss: 0.0237 - val_g_loss: 1.4932e-04 - val_d_loss: 1.5532e-04
Epoch 2/50
411/411 [==============================] - 6s 14ms/step - g_loss: 1.0371e-04 - d_loss: 1.0904e-04 - val_g_loss: 1.2473e-04 - val_d_loss: 1.2472e-04
Epoch 3/50
411/411 [==============================] - 6s 13ms/step - g_loss: 4.2017e-05 - d_loss: 4.2923e-05 - val_g_loss: 1.0039e-04 - val_d_loss: 9.9799e-05
Epoch 4/50
411/411 [==============================] - 5s 13ms/step - g_loss: 2.5003e-05 - d_loss: 2.5336e-05 - val_g_loss: 8.4076e-05 - val_d_loss: 8.4437e-05
Epoch 5/50
411/411 [==============================] - 6s 13ms/step - g_loss: 2.2526e-05 - d_loss: 2.2641e-05 - val_g_loss: 7.2394e-05 - val_d_loss: 7.2784e-05
Epoch 6/50
411/411 [==============================] - 6s 14ms/step - g_loss: 3.0002e-05 - d_loss: 3.0121e-05 - val_g_loss: 6.5266e-05 - val_d_loss: 6.5329e-05
Epoch 7/50
411/411 [==============================] - 

In [13]:
# tcnusad_times, _ = tcnusad.fit(train_dataset, val_dataset, epochs)
# print(f'TCN-USAD train time: {sum(tcnusad_times):.4f}')
tcnusad.fit(train_dataset, epochs=epochs, batch_size=batch_size, validation_data=(val_dataset), callbacks=[tcnusad_time])
print(f'TCN-USAD train time: {sum(tcnusad_time.times):.4f}')

Epoch 1/50
411/411 [==============================] - 9s 16ms/step - g_loss: 0.0223 - d_loss: 0.0223 - val_g_loss: 0.0062 - val_d_loss: 0.0064
Epoch 2/50
411/411 [==============================] - 6s 15ms/step - g_loss: 0.0041 - d_loss: 0.0041 - val_g_loss: 0.0059 - val_d_loss: 0.0062
Epoch 3/50
411/411 [==============================] - 6s 15ms/step - g_loss: 0.0031 - d_loss: 0.0032 - val_g_loss: 0.0051 - val_d_loss: 0.0054
Epoch 4/50
411/411 [==============================] - 6s 15ms/step - g_loss: 0.0021 - d_loss: 0.0021 - val_g_loss: 0.0047 - val_d_loss: 0.0049
Epoch 5/50
411/411 [==============================] - 6s 15ms/step - g_loss: 0.0017 - d_loss: 0.0017 - val_g_loss: 0.0042 - val_d_loss: 0.0045
Epoch 6/50
411/411 [==============================] - 6s 16ms/step - g_loss: 0.0013 - d_loss: 0.0013 - val_g_loss: 0.0038 - val_d_loss: 0.0040
Epoch 7/50
411/411 [==============================] - 6s 16ms/step - g_loss: 0.0012 - d_loss: 0.0012 - val_g_loss: 0.0034 - val_d_loss: 0.0036

In [14]:
# To-Do
# 00. make TCN-USAD(Done)
# 00. train USAD with keras fit
# 01. prepare dataset(LEAD 1.0, EPIC, aihub)(ing)
# 02. test and make a Smart Media Journal Paper

# # 모델 가중치 학습 가능 여부 확인 코드
# for weight in autoencoder.encoder.trainable_weights:
#     print(f"Weight name: {weight.name}, Trainable: {weight.trainable}")
# for weight in autoencoder.decoder.trainable_weights:
#     print(f"Weight name: {weight.name}, Trainable: {weight.trainable}")
# autoencoder.encoder.trainable_weights
# autoencoder.decoder.trainable_weights

In [15]:
# # save model weights
# path = './save/JNU/'
# check_path(path)
# lstmae.save(e_path=path+'LSTM_Encoder.h5', d_path=path+'LSTM_Decoder.h5')
# gruae.save(e_path=path+'GRU_Encoder.h5', d_path=path+'GRU_Decoder.h5')
# tcnae_s.save(e_path=path+'TCN_Encoder_s.h5', d_path=path+'TCN_Decoder_s.h5')
# tcnae_l.save(e_path=path+'TCN_Encoder_l.h5', d_path=path+'TCN_Decoder_l.h5')

In [16]:
# path = './save/JNU/'
# lstmae.load(e_path=path+'LSTM_Encoder.h5', d_path=path+'LSTM_Decoder.h5')